# Background Mask
Create a background mask for the invivo-imaging pipeline based on ground truth neuron masks.

In [ ]:
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from pathlib import Path
import sys
sys.path.append(str(Path('../../evaluate').absolute()))
from roi import read_roi

## Specify Dataset and Neuron Masks

In [ ]:
INPUT_PATH = '/media/bandy/nvme_work/voltage/compare/invivo/lowmag/'
GT_PATH = '/media/bandy/nvme_data/voltage/datasets_v0.5/lowmag_GT'
DATASET_NAME = '02_02'

In [ ]:
input_file = Path(INPUT_PATH, DATASET_NAME, 'Sn_image.tif')
im = tiff.imread(input_file).astype(float).squeeze()

gt_file = Path(GT_PATH, DATASET_NAME + '.tif')
gt = read_roi(gt_file, (0,))
mask = np.logical_or.reduce(gt)
ofs_y = (mask.shape[0] - im.shape[0]) // 2
ofs_x = (mask.shape[1] - im.shape[1]) // 2
mask = mask[ofs_y:ofs_y + im.shape[0], ofs_x:ofs_x + im.shape[1]]

plt.figure(figsize=(11, 5))
plt.subplot(1, 2, 1)
plt.axis('off')
plt.imshow(im, cmap='gray')
plt.subplot(1, 2, 2)
plt.axis('off')
plt.imshow(mask, cmap='gray')
plt.show()

## Create Background Mask
By dilating and inverting the neuron masks.
Change the dilation radius and further modify the code as necessary. 

In [ ]:
from skimage.morphology import disk, binary_dilation
radius = 10
foreground = binary_dilation(mask, disk(radius, dtype=bool))
background = ~foreground

plt.figure(figsize=(11, 5))
plt.subplot(1, 2, 1)
plt.axis('off')
plt.imshow(background, cmap='gray')
plt.subplot(1, 2, 2)
plt.axis('off')
plt.imshow(np.multiply(im, background), cmap='gray')
plt.show()

tiff.imwrite(input_file.with_name('background.tif'), background, photometric='minisblack')